In [7]:
import pyomo.environ as pyo
import pandas as pd
import math
import time

start_time = time.time()

clientes_df = pd.read_csv("clients.csv")
depot_df    = pd.read_csv("depots.csv")
veh_df      = pd.read_csv("vehicles.csv")
params_df   = pd.read_csv("parameters_base.csv")

# Configuración básica
DEPOT = 0
depot_lat  = depot_df["Latitude"].iloc[0]
depot_lon  = depot_df["Longitude"].iloc[0]
coords = {0: (depot_lat, depot_lon)}
CLIENTES = clientes_df["ClientID"].tolist()

print(f"Problema con {len(CLIENTES)} clientes y {len(veh_df)} vehículos")

for _, row in clientes_df.iterrows():
    coords[row["ClientID"]] = (row["Latitude"], row["Longitude"])

demand = {row["ClientID"]: row["Demand"] for _, row in clientes_df.iterrows()}

# Vehículos
V = veh_df["VehicleID"].tolist()
rv = {row["VehicleID"]: row["Capacity"] for _, row in veh_df.iterrows()}
tv = {row["VehicleID"]: row["Range"] for _, row in veh_df.iterrows()}

# Parámetros
fuel_price = params_df.loc[params_df["Parameter"] == "fuel_price", "Value"].iloc[0]
fuel_eff   = params_df.loc[params_df["Parameter"] == "fuel_efficiency_typical", "Value"].iloc[0]
ev = {v: fuel_eff for v in V}

# Costos 
co = 50000     # costo operativo por vehículo
pf = fuel_price # precio del combustible
# ct = 0, cm = 0 según el base case

# Función Haversine
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Matriz de distancias
NODOS = [DEPOT] + CLIENTES
w = {}
for i in NODOS:
    for j in NODOS:
        if i == j:
            w[i, j] = 0.0
        else:
            w[i, j] = haversine_km(coords[i][0], coords[i][1], coords[j][0], coords[j][1])


# modelo pyomo
model = pyo.ConcreteModel()

# Conjuntos
model.N = pyo.Set(initialize=NODOS)
model.C = pyo.Set(initialize=CLIENTES) 
model.V = pyo.Set(initialize=V)
model.D = pyo.Set(initialize=[DEPOT])

# Parámetros
model.w = pyo.Param(model.N, model.N, initialize=w, within=pyo.NonNegativeReals)
model.q = pyo.Param(model.C, initialize=demand, within=pyo.NonNegativeReals)
model.r = pyo.Param(model.V, initialize=rv, within=pyo.NonNegativeReals)
model.tau = pyo.Param(model.V, initialize=tv, within=pyo.NonNegativeReals)
model.e = pyo.Param(model.V, initialize=ev, within=pyo.PositiveReals)

model.co = pyo.Param(initialize=co)
model.pf = pyo.Param(initialize=pf)

# Costo por arco 
def s_init(model, i, j, v):
    return (pf / model.e[v]) * w[i, j]  # Solo costo de combustible
model.s = pyo.Param(model.N, model.N, model.V, initialize=s_init, within=pyo.NonNegativeReals)

# Variables
model.x = pyo.Var(model.N, model.N, model.V, domain=pyo.Binary)
model.y = pyo.Var(model.V, domain=pyo.Binary)

# Eliminar bucles
def no_self_arcs_rule(m, i, v):
    return m.x[i, i, v] == 0
model.no_self_arcs = pyo.Constraint(model.N, model.V, rule=no_self_arcs_rule)

# Objetivo
def obj_rule(m):
    return sum(m.s[i, j, v] * m.x[i, j, v] for i in m.N for j in m.N for v in m.V) + sum(m.co * m.y[v] for v in m.V)
model.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Restricciones BÁSICAS
def flow_cons_rule(m, v, c):
    return sum(m.x[i, c, v] for i in m.N if i != c) - sum(m.x[c, j, v] for j in m.N if j != c) == 0
model.flow_cons = pyo.Constraint(model.V, model.C, rule=flow_cons_rule)

def autonomy_rule(m, v):
    return sum(m.x[i, j, v] * m.w[i, j] for i in m.N for j in m.N if i != j) <= m.tau[v]
model.autonomy = pyo.Constraint(model.V, rule=autonomy_rule)

# Restricción de capacidad SIMPLIFICADA
def capacity_rule(m, v):
    return sum(m.q[c] * sum(m.x[i, c, v] for i in m.N if i != c) for c in m.C) <= m.r[v] * m.y[v]
model.capacity = pyo.Constraint(model.V, rule=capacity_rule)

def depart_from_depot_rule(m, v):
    return sum(m.x[DEPOT, j, v] for j in m.C) == m.y[v]
model.depart_depot = pyo.Constraint(model.V, rule=depart_from_depot_rule)

def return_to_depot_rule(m, v):
    return sum(m.x[i, DEPOT, v] for i in m.C) == m.y[v]
model.return_depot = pyo.Constraint(model.V, rule=return_to_depot_rule)

def cover_once_rule(m, j):
    return sum(m.x[i, j, v] for v in m.V for i in m.N if i != j) == 1
model.cover_once = pyo.Constraint(model.C, rule=cover_once_rule)

solver = pyo.SolverFactory('glpk')

# Configurar límites más relajados para mejor solución
solver_options = {
    'tmlim': 120,  # 2 minutos máximo
    'mipgap': 0.05  # 5% de gap (más estricto)
}

result = solver.solve(model, options=solver_options, tee=True)

print(f"Tiempo de solución: {time.time() - start_time:.2f} segundos")


if result.solver.termination_condition in [pyo.TerminationCondition.optimal, pyo.TerminationCondition.feasible]:
    
    if result.solver.termination_condition == pyo.TerminationCondition.optimal:
        print("Solucion encontrada (optima)")
    else:
        print("solucion factible encontrada (no necesariamente óptima)")
        print(f"  Gap: {result.solver.relative_gap if hasattr(result.solver, 'relative_gap') else 'N/A'}")
    
    # Análisis de resultados
    costo_combustible = 0
    costo_fijo = 0
    rutas_vehiculos = {}
    vehiculos_utilizados = 0

    for v in model.V:
        y_val = pyo.value(model.y[v])
        if y_val > 0.5:
            vehiculos_utilizados += 1
            costo_fijo += co
            
            # Reconstruir ruta
            route = ["CD0"]
            current = DEPOT
            visited = set()
            
            for _ in range(len(NODOS) + 2):
                found_next = False
                for j in NODOS:
                    if j != current:
                        x_val = pyo.value(model.x[current, j, v])
                        if x_val is not None and x_val > 0.5:
                            if j == DEPOT:
                                route.append("CD0")
                                current = j
                                found_next = True
                                break
                            else:
                                route.append(f"C{j}")
                                visited.add(j)
                                current = j
                                found_next = True
                                break
                if not found_next or current == DEPOT:
                    break
            
            # Calcular métricas
            distancia_ruta = 0
            carga_ruta = 0
            costo_vehiculo = 0
            
            for i in range(len(route) - 1):
                node_i = DEPOT if route[i] == "CD0" else int(route[i][1:])
                node_j = DEPOT if route[i + 1] == "CD0" else int(route[i + 1][1:])
                distancia_ruta += w[node_i, node_j]
                costo_tramo = (pf / ev[v]) * w[node_i, node_j]
                costo_vehiculo += costo_tramo
                costo_combustible += costo_tramo
            
            for client in visited:
                carga_ruta += demand[client]
            
            rutas_vehiculos[v] = {
                'ruta': route,
                'distancia': distancia_ruta,
                'carga': carga_ruta,
                'clientes': visited,
                'costo_combustible': costo_vehiculo
            }

    # Mostrar resultados detallados
    costo_total = costo_combustible + costo_fijo
    
    print(f"\nResumen")
    print(f"Vehículos utilizados: {vehiculos_utilizados} de {len(V)}")
    print(f"Costo combustible: {costo_combustible:,.0f} COP")
    print(f"Costo fijo: {costo_fijo:,.0f} COP")
    print(f"Costo total: {costo_total:,.0f} COP")
    print(f"Total demanda: {sum(demand.values()):,.0f} kg")
    
    print(f"\nrutas:")
    for v, datos in rutas_vehiculos.items():
        print(f"\nVehículo {v} (Capacidad: {rv[v]:,.0f}kg, Autonomía: {tv[v]:,.0f}km):")
        print(f"  Ruta: {' → '.join(datos['ruta'])}")
        print(f"  Distancia: {datos['distancia']:.1f} km / {tv[v]:.0f} km")
        print(f"  Carga: {datos['carga']:,.0f} kg / {rv[v]:,.0f} kg")
        print(f"  Clientes: {sorted([f'C{c}' for c in datos['clientes']])}")
        print(f"  Costo combustible: {datos['costo_combustible']:,.0f} COP")
    
    # Métricas de eficiencia
    distancia_total = sum(datos['distancia'] for datos in rutas_vehiculos.values())
    print(f"\nMetricas:")
    print(f"Distancia total recorrida: {distancia_total:.1f} km")
    print(f"Costo por km: {costo_combustible/distancia_total:,.0f} COP/km" if distancia_total > 0 else "N/A")
    print(f"Clientes por vehículo: {len(CLIENTES)/vehiculos_utilizados:.1f}" if vehiculos_utilizados > 0 else "N/A")
    
else:
    print(f" No se encontró solución: {result.solver.termination_condition}")

Problema con 24 clientes y 8 vehículos
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 120 --mipgap 0.05 --write C:\Users\Andres\AppData\Local\Temp\tmp2ruzz0km.glpk.raw
 --wglp C:\Users\Andres\AppData\Local\Temp\tmpumf4gmfa.glpk.glp --cpxlp C:\Users\Andres\AppData\Local\Temp\tmp5f0abxj2.pyomo.lp
Reading problem data from 'C:\Users\Andres\AppData\Local\Temp\tmp5f0abxj2.pyomo.lp'...
C:\Users\Andres\AppData\Local\Temp\tmp5f0abxj2.pyomo.lp:35010: warning: lower bound of variable 'x4811' redefined
C:\Users\Andres\AppData\Local\Temp\tmp5f0abxj2.pyomo.lp:35010: warning: upper bound of variable 'x4811' redefined
448 rows, 5008 columns, 23840 non-zeros
5008 integer variables, all of which are binary
40018 lines were read
Writing problem data to 'C:\Users\Andres\AppData\Local\Temp\tmpumf4gmfa.glpk.glp'...
34147 lines were written
GLPK Integer Optimizer 5.0
448 rows, 5008 columns, 23840 non-zeros
5008 integer variables, all of which are binary
Preprocessing...
